# Processing
Demonstration of Processing endpoints.

In [1]:
import utils.DemoClient as client
import jwt
import json

## Initialise

### Client
We instantiate a client to interact with the platform. The client dynamically registers with the Authorisation Server to take part in UMA (User Managed Access) flows through which authorization is obtained for scoped access resources on behalf of the user.

In [4]:
#-------------------------------------------------------------------------------
# Initialise client
#-------------------------------------------------------------------------------
base_domain = "develop.eoepca.org"
platform_domain = "auth." + base_domain
base_url = "https://" + platform_domain
demo = client.DemoClient(base_url)
demo.register_client()
demo.save_state()

client_id: b8313201-4b57-4b04-af60-0c7876005ea1
Client state saved to file: state.json


### User
User authenticates and the client receives an ID Token (JWT) that represents the user.

In [5]:
#-------------------------------------------------------------------------------
# Authenticate as UserA and get ID Token
#-------------------------------------------------------------------------------
USER_NAME="eric"
USER_PASSWORD="defaultPWD"
user_id_token = demo.get_id_token(USER_NAME, USER_PASSWORD)

[Request] GET => https://auth.develop.eoepca.org/.well-known/uma2-configuration
<Response [200]>
token_endpoint: https://auth.develop.eoepca.org/oxauth/restv1/token
[Request] POST => https://auth.develop.eoepca.org/oxauth/restv1/token


## ADES
The ADES provides user-specific endpoints, using a URL path prefix.

In [6]:
# ADES URLs
ades_base_url = "http://ades.develop.eoepca.org"  #"http://ades." + platform_domain
ades_wps_url = ades_base_url + "/" + USER_NAME + "/zoo"; print("ADES WPS endpoint:", ades_wps_url)
ades_proc_url = ades_base_url + "/" + USER_NAME + "/wps3"; print("ADES API Processes endpoint:", ades_proc_url)

ADES WPS endpoint: http://ades.develop.eoepca.org/eric/zoo
ADES API Processes endpoint: http://ades.develop.eoepca.org/eric/wps3


### ADES API Processes

In [7]:
ades_access_token = None
app_name = "dnbr"

#### ADES: List Processes

In [8]:
# API Processes - List Processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

[Request] GET => http://ades.develop.eoepca.org/eric/wps3/processes


UnboundLocalError: local variable 'ticket' referenced before assignment

#### ADES: Deploy Application

In [ ]:
# API Processes - Deploy Application
response, ades_access_token = demo.proc_deploy_application(ades_proc_url, "../data/dnbr/app-deploy-body-cwl.json", id_token=user_id_token, access_token=ades_access_token)

In [ ]:
# Check deploy - list processes again
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

#### ADES: Get Application Details

In [ ]:
# API Processes - Get Application Details
response, ades_access_token = demo.proc_get_app_details(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

#### ADES: Execute Application

In [ ]:
# API Processes - Execute Application
response, ades_access_token, job_location_path = demo.proc_execute_application(ades_proc_url, app_name, "../data/snuggs-0.3.0/app-execute-body-single.json", id_token=user_id_token, access_token=ades_access_token)

#### ADES: Job Status

In [ ]:
# API Processes - Job Status
response, ades_access_token, status = demo.proc_get_job_status(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

In [ ]:
# API Processes - Job Status (keep polling for completion)
response, ades_access_token, status = demo.proc_poll_job_completion(ades_base_url, job_location_path, interval=10, id_token=user_id_token, access_token=ades_access_token)

In [ ]:
print(response.status_code, response.headers, response.text)

#### ADES: Job Result

In [ ]:
# API Processes - Job Result
response, ades_access_token, stacCatalogUri = demo.proc_get_job_result(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

#### ADES: List Jobs

In [ ]:
# API Processes - List Jobs
response, ades_access_token, job_ids = demo.proc_list_jobs(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
print("Jobs:", job_ids)

#### ADES: Undeploy Application

In [ ]:
# API Processes - Undeploy Application
response, ades_access_token = demo.proc_undeploy_application(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)

In [ ]:
# Check undeploy - list processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

## Workspace

In [ ]:
workspace_url = "https://workspace-api." + base_domain
workspace_access_token = None

### Workspace: Get Details

In [ ]:
# Workspace - Get Details
workspace_name = "rm-user-" + USER_NAME.lower()
response, workspace_access_token = demo.workspace_get_details(workspace_url, workspace_name, id_token=user_id_token, access_token=workspace_access_token)
workspace_details = response.json()
print(json.dumps(workspace_details, indent = 2))

### Inspect S3 Bucket

In [ ]:
# Bucket details
bucket_name = workspace_details["storage"]["credentials"]["bucketname"]
s3_access = workspace_details["storage"]["credentials"]["access"]
s3_secret = workspace_details["storage"]["credentials"]["secret"]

In [ ]:
# Quick hack S3 access
import boto3

# Init S3 session for Creodias
S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)

# List bucket contents
bucket = s3resource.Bucket(bucket_name)
for obj in bucket.objects.all():2021-4162681
    print(' ->', obj)